In [1]:
# %load ~/ipyhead
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns


We're going to load quantification data from Kallisto. We used the index: `Homo_sapiens.GRCh38.cdna.all.kallisto.idx`

Per http://f1000research.com/articles/4-1521/v2 (https://github.com/mikelove/tximport), we should use TPM (not estimated counts, which are sensitive to transcript length) and aggregate to gene level.

Here is how to annotate with gene names in Python: https://github.com/hammerlab/cohorts/blob/master/cohorts/load.py#L797-L801

We made the index by downloading from http://ftp.ensembl.org/pub/release-79/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz , which is the same as what's suggested on kallisto website. So use release 79.

Though note that the newest release is release 85, from ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/README_ensembl:

```
#tax_id	ncbi_release	ncbi_assembly	ensembl_release	ensembl_assembly	date_compared
9606	Homo sapiens Annotation Release 108	GRCh38.p7	85	GRCh38.p7	20160801
```

And http://ftp.ensembl.org/pub/release-85/fasta/homo_sapiens/cdna/ has different files.

In [2]:
df = pd.read_csv('output/ERR431617/abundance.tsv', sep='\t')
df.head()

,target_id,length,eff_length,est_counts,tpm
0,ENST00000415118,8,5.08333,0.0,0.00000
1,ENST00000448914,13,8.66667,0.0,0.00000
2,ENST00000434970,9,5.69231,0.0,0.00000
3,ENST00000390577,37,21.37140,1.0,6.44565
4,ENST00000437320,19,11.90480,0.0,0.00000


In [3]:
!pip install pyensembl

    100% |████████████████████████████████| 61kB 1.6MB/s 
    100% |████████████████████████████████| 14.3MB 74kB/s 
    100% |████████████████████████████████| 81kB 9.5MB/s 
  Running setup.py bdist_wheel for pyensembl ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/04/69/38/f675315f843ae0d30475fbbdd865622995dc64ae7adee83a3f
  Running setup.py bdist_wheel for typechecks ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/f5/9e/66/87ce0b0e4abaf030ccdf2822cdd59cc2b1fffa6584236faf5e
  Running setup.py bdist_wheel for datacache ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/f6/37/55/ba59c1f1e2c7df84eca3df207aba1f9ed385bc4d2f674db8d2
  Running setup.py bdist_wheel for memoized-property ... - done
  Stored in directory: /home/maxim/.cache/pip/wheels/09/42/c6/fa081008bb537f37b06168cb296c94d70f836c49d1f52c78ba
  Running setup.py bdist_wheel for gtfparse ... - \ done
  Stored in directory: /home/maxim/.cache/pip/wheels/ed/90/

In [4]:
!pyensembl install --release 79 --species human

-- Running 'install' for EnsemblRelease(release=79, species='homo_sapiens')
INFO:root:Fetching /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.79.gtf.gz from URL ftp://ftp.ensembl.org/pub/release-79/gtf/homo_sapiens/Homo_sapiens.GRCh38.79.gtf.gz
INFO:root:Fetching /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-79/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:root:Fetching /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.pep.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-79/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz
Creating database: /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.79.gtf.db
Reading GTF from /home/maxim/.cache/pyensembl/GRCh38/ensembl79/Homo_sapiens.GRCh38.79.gtf.gz
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_vers

In [5]:
from pyensembl import cached_release

In [6]:
ensembl_release = cached_release(79)
df['sample_id'] = 1
df['gene_name'] = df['target_id'].map(lambda t: ensembl_release.gene_name_of_transcript_id(t))

In [29]:
# sum counts, abundance (tpm) across genes
simple_summary = df.groupby(['sample_id', 'gene_name'])[['est_counts', 'tpm']].sum().reset_index()
simple_summary.to_csv('summary.simple.tsv', sep='\t', index=None)
simple_summary.head()

,sample_id,gene_name,est_counts,tpm
0,1,A1BG,89.513800,14.958630
1,1,A1CF,14.071500,0.249757
2,1,A2M,0.000000,0.000000
3,1,A2ML1,16.530498,1.209676
4,1,A2MP1,2.000000,0.076626


Scale the TPM, as in that paper:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173259 entries, 0 to 173258
Data columns (total 7 columns):
target_id     173259 non-null object
length        173259 non-null int64
eff_length    173259 non-null float64
est_counts    173259 non-null float64
tpm           173259 non-null float64
sample_id     173259 non-null int64
gene_name     173259 non-null object
dtypes: float64(3), int64(2), object(2)
memory usage: 9.3+ MB


In [12]:
# eff_length is the right one to use.
any(pd.isnull(df.length)), any(pd.isnull(df.eff_length))

(False, False)

How to summarize tpm, from https://github.com/mikelove/tximport/blob/master/R/tximport.R

abundanceMat is from tpm. lengthMat is from eff_length. countsMat is from est_counts. These are gene-level.
Any mat Tx is transcript-level.

```
  # summarize abundance and counts
  message("summarizing abundance")
  abundanceMat <- rowsum(abundanceMatTx, geneId)
  message("summarizing counts")
  countsMat <- rowsum(countsMatTx, geneId)
  message("summarizing length")
  
  # the next lines calculate a weighted average of transcript length, 
  # weighting by transcript abundance.
  # this can be used as an offset / normalization factor which removes length bias
  # for the differential analysis of estimated counts summarized at the gene level.
  weightedLength <- rowsum(abundanceMatTx * lengthMatTx, geneId)
  lengthMat <- weightedLength / abundanceMat   

  # pre-calculate a simple average transcript length
  # for the case the abundances are all zero for all samples.
  # first, average the tx lengths over samples
  aveLengthSamp <- rowMeans(lengthMatTx)
  # then simple average of lengths within genes (not weighted by abundance)
  aveLengthSampGene <- tapply(aveLengthSamp, geneId, mean)

  stopifnot(all(names(aveLengthSampGene) == rownames(lengthMat)))
  
  # check for NaN and if possible replace these values with geometric mean of other samples.
  # (the geometic mean here implies an offset of 0 on the log scale)
  # NaN come from samples which have abundance of 0 for all isoforms of a gene, and 
  # so we cannot calculate the weighted average. our best guess is to use the average
  # transcript length from the other samples.
  lengthMat <- replaceMissingLength(lengthMat, aveLengthSampGene)
  
  if (countsFromAbundance != "no") {
    countsSum <- colSums(countsMat)
    if (countsFromAbundance == "lengthScaledTPM") {
      newCounts <- abundanceMat * rowMeans(lengthMat)
    } else {
        newCounts <- abundanceMat
      }
    newSum <- colSums(newCounts)
    countsMat <- t(t(newCounts) * (countsSum/newSum))
  }
    return(list(abundance=abundanceMat, counts=countsMat, length=lengthMat,
              countsFromAbundance=countsFromAbundance))


# function for replacing missing average transcript length values
replaceMissingLength <- function(lengthMat, aveLengthSampGene) {
  nanRows <- which(apply(lengthMat, 1, function(row) any(is.nan(row))))
  if (length(nanRows) > 0) {
    for (i in nanRows) {
      if (all(is.nan(lengthMat[i,]))) {
        # if all samples have 0 abundances for all tx, use the simple average
        lengthMat[i,] <- aveLengthSampGene[i]
      } else {
          # otherwise use the geometric mean of the lengths from the other samples
          idx <- is.nan(lengthMat[i,])
          lengthMat[i,idx] <-  exp(mean(log(lengthMat[i,!idx]), na.rm=TRUE))
        }
    }
  }
  lengthMat
}


```

Note how the matrices are set up:

* rows: transcripts (n)
* columns: samples (m)

In R:
* rowSums returns n*1 -- i.e. sums columns in every row
* colSums return 1*m: i.e. sums all rows for every column
* similarly, rowMeans returns n*1: takes mean of values in each row.
* similarly, colMeans returns m*1: takes mean of values in each column.

```
> x <- cbind(x1 = 3, x2 = c(4:1, 2:5))
> x
     x1 x2
[1,]  3  4
[2,]  3  3
[3,]  3  2
[4,]  3  1
[5,]  3  2
[6,]  3  3
[7,]  3  4
[8,]  3  5
> rowSums(x)
[1] 7 6 5 4 5 6 7 8
> colSums(x)
x1 x2 
24 24 
> rowMeans(x)
[1] 3.5 3.0 2.5 2.0 2.5 3.0 3.5 4.0
> colMeans(x)
x1 x2 
 3  3 
```

but:

* rowsum(x, group) means take vertical (column) sums across rows for each value of a grouping variable. returns k (number of groups) * m.

```
> require(stats)
> x <- matrix(runif(100), ncol = 5)
> x
            [,1]       [,2]        [,3]       [,4]       [,5]
 [1,] 0.92193476 0.64068964 0.590930078 0.22589474 0.94385005
 [2,] 0.15434593 0.35520130 0.891485406 0.01536309 0.23128203
 [3,] 0.65030467 0.56747106 0.583187841 0.50184454 0.25598191
 [4,] 0.08740973 0.77858838 0.001003955 0.09488344 0.40001838
 [5,] 0.59932257 0.10921583 0.202659336 0.99391111 0.05304302
 [6,] 0.37518234 0.15261407 0.201251270 0.39387585 0.06438196
 [7,] 0.64699723 0.81092956 0.888885059 0.91184237 0.68612899
 [8,] 0.83465129 0.23701509 0.182450580 0.94085281 0.55303249
 [9,] 0.67748311 0.39298104 0.743816100 0.04000909 0.04451748
[10,] 0.36702365 0.51603955 0.676513819 0.21491027 0.97969625
[11,] 0.81330661 0.01567229 0.265442836 0.33373162 0.03803008
[12,] 0.42658481 0.73068255 0.072104913 0.64861247 0.49143138
[13,] 0.68161538 0.11003679 0.066483887 0.95039580 0.78276828
[14,] 0.57063695 0.85563269 0.517796875 0.11461768 0.36473195
[15,] 0.89833696 0.53448586 0.053799714 0.35298232 0.71251391
[16,] 0.93033111 0.98725333 0.743280945 0.70274502 0.91984371
[17,] 0.72468190 0.80205415 0.304485959 0.51483852 0.03668938
[18,] 0.29001996 0.60153044 0.976458703 0.72192388 0.14789380
[19,] 0.46333744 0.25058092 0.471588748 0.05430856 0.10111760
[20,] 0.78955687 0.94806861 0.675796005 0.87945982 0.77918547
> group <- sample(1:8, 20, TRUE)
> group
 [1] 5 4 3 5 5 7 8 5 6 4 2 4 5 5 1 4 4 3 3 3
> (xsum <- rowsum(x, group))
       [,1]       [,2]       [,3]       [,4]       [,5]
1 0.8983370 0.53448586 0.05379971 0.35298232 0.71251391
2 0.8133066 0.01567229 0.26544284 0.33373162 0.03803008
3 2.1932189 2.36765104 2.70703130 2.15753680 1.28417877
4 2.6029674 3.39123087 2.68787104 2.09646937 2.65894275
5 3.6955707 2.73117843 1.56132471 3.32055558 3.09744417
6 0.6774831 0.39298104 0.74381610 0.04000909 0.04451748
7 0.3751823 0.15261407 0.20125127 0.39387585 0.06438196
8 0.6469972 0.81092956 0.88888506 0.91184237 0.68612899
```

In [138]:
# reimplement R, but keep in tidy form: samples are not columns anymore, we have sample ID column instead

abundanceMat = df.groupby(['sample_id', 'gene_name'])[['tpm']].sum()
df_tmp = df.copy()
df_tmp['weightedLength'] = df_tmp['tpm'] * df_tmp['eff_length']
weightedLength = df_tmp.groupby(['sample_id', 'gene_name'])[['weightedLength']].sum()
lengthMat = (weightedLength['weightedLength']/ abundanceMat['tpm']).reset_index()
# make wide again: make the samples be columns. n_genes x n_samples, values are lengths
lengthMat = lengthMat.pivot( index='gene_name', columns='sample_id', values=0)

## replace missing average transcript length values

# first, calculate average transcript length *across samples* for each transcript
# but keep gene name in there
aveLengthSamp = df.groupby('target_id')[['eff_length', 'gene_name']].agg({
        'eff_length': 'mean',
        'gene_name': 'first'
    })
# second, take simple average of lengths within genes, without weighting by abundance
# i.e. aggregate those averages by gene now
aveLengthSampGene = aveLengthSamp.groupby('gene_name').eff_length.mean()

# do the replacement
nanRows = lengthMat[lengthMat.isnull().any(axis=1)].index # np.where(pd.isnull(lengthMat))[0] # row indices where null
for row_idx in nanRows:
    if all(pd.isnull(lengthMat.loc[row_idx])):
        # use simple average
        lengthMat.loc[row_idx] = aveLengthSampGene.loc[row_idx] # lengthMat.index[row_idx] # look up by gene name
    else:
        # use geometric mean of lengths from other samples
        null_indices = np.where(pd.isnull(lengthMat.loc[row_idx]))[0]
        others = np.where(~pd.isnull(lengthMat.loc[row_idx]))[0]
        lengthMat.loc[row_idx, null_indices] = np.exp(np.mean(np.log(lengthMat.loc[row_idx, others])))
assert not pd.isnull(lengthMat).any().bool()

In [ ]:
# length scaled TPM
counts_LS = abundanceMat


In [23]:
# either scaledTPM or lengthScaledTPM

def scaledTPM(matrix):
    
    pass
def lengthScaledTPM(matrix):
    pass

def core(matrix):
    pass
abundanceMat = df.groupby(['sample_id', 'gene_name'])[['tpm']].sum()#.reset_index()
df_tmp = df.copy()
df_tmp['weightedLength'] = df_tmp['tpm'] * df_tmp['eff_length']
weightedLength = df_tmp.groupby(['sample_id', 'gene_name'])[['weightedLength']].sum()#.reset_index()
lengthMat = (weightedLength['weightedLength']/ abundanceMat['tpm']).reset_index()

# replace missing average transcript length values
# first, calculate average transcript length *across samples*


# replace for case where abundances are all 0 for all samples

# function for replacing missing average transcript length values
def replaceMissingLength(df, aveLengthSampGene):
    lengthMat = df.copy()
    nanRows = np.where(pd.isnull(lengthMat))[0] # row indices where null
    for row_idx in nanRows:
        if all(pd.isnull(lengthMat[row_idx])):
            # use simple average
            lengthMat.loc[row_idx] = aveLengthSampGene[row_idx]
        else:
            # use geometric mean of lengths from other samples
            null_indices = np.where(pd.isnull(lengthMat[row_idx]))
            others = np.where(~pd.isnull(lengthMat[row_idx]))
            lengthMath.loc[row_idx, null_indices] = np.exp(np.mean(np.log(lengthMat[row_idx, others])))
        
    return lengthMat

(                        weightedLength
 sample_id gene_name                   
 1         A1BG            12330.780428
           A1CF             1938.386555
           A2M                 0.000000
           A2ML1            2277.117735
           A2MP1             275.504952
           A3GALT2           413.259230
           A4GALT            688.761456
           A4GNT               0.000000
           AAAS             4683.586265
           AACS            22349.414328
           AACSP1           1229.235312
           AADAC               0.000000
           AADACL2             0.000000
           AADACL3           275.505086
           AADACL4           413.256817
           AADACP1             0.000000
           AADAT             137.752799
           AAED1           41436.691593
           AAGAB           64055.062744
           AAK1            21099.125589
           AAMDC            5703.541360
           AAMP            47800.183318
           AANAT            4008.740330


In [26]:
np.where(pd.isnull(lengthMat[2]))

KeyError: 2